# Run the CICI SAFE Client

#### Configure the Envrionment

In [1]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
#export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
#export OS_REGION_NAME='CHI@TACC'
#export OS_REGION_NAME='CHI@UC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter

export CLIENT_NAME=client1
# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
#export CLIENT_RESERVATION_ID='8882c0ea-e92a-404e-8637-73b4c38f4101'

#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter-Tridentcom
export UNIVERSAL_INFO_FILE=${WORKING_DIR}/universal_info
export CLIENT_INFO_FILE=${WORKING_DIR}/${CLIENT_NAME}_info

#Set GENI CERT file
export GENI_CERT=${HOME}/work/geni-pruth.pem

export CLIENT_NETWORK='sharednet1'
export CLIENT_NODE_NAME=${USERNAME}-${CLIENT_NAME}
export CLIENT_STACK_NAME=${USERNAME}-${CLIENT_NAME}

#SDX Info
export SDX_INFO_FILE=${WORKING_DIR}/sdx_info
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
echo SDX_PUBLIC_IP: $SDX_PUBLIC_IP

#RIAK Info
export RIAK_INFO_FILE=${WORKING_DIR}/riak_info
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`
echo RIAK_PUBLIC_IP: $RIAK_PUBLIC_IP

export OS_REGION_NAME=`cat $UNIVERSAL_INFO_FILE | grep OS_REGION_NAME | cut -d " " -f 2`
export OS_PROJECT_NAME=`cat $UNIVERSAL_INFO_FILE | grep OS_PROJECT_NAME | cut -d " " -f 2`
echo OS_REGION_NAME $OS_REGION_NAME
echo OS_PROJECT_NAME $OS_PROJECT_NAME


SDX_PUBLIC_IP: 192.5.87.155
RIAK_PUBLIC_IP: 192.5.87.143
OS_REGION_NAME CHI@UC
OS_PROJECT_NAME CH-816532


In [2]:
#Clear and rebuild Client File
echo "" > $CLIENT_INFO_FILE

echo CLIENT_NODE_NAME $CLIENT_NODE_NAME >> $CLIENT_INFO_FILE
echo CLIENT_STACK_NAME $CLIENT_STACK_NAME >> $CLIENT_INFO_FILE

#### Create Lease (if necessary)

In [3]:
#Check of universal reservation
if [ -f "${UNIVERSAL_INFO_FILE}" ]; then
   LEASE_NAME=`cat $UNIVERSAL_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
   echo $LEASE_NAME
   echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE
fi

if  [ -z $LEASE_NAME ]; then
  
    echo No universal reservation. Creating new reservation

# --start-date "2019-02-10 17:50" \
# --end-date "2019-02-11 12:00" \
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`
LEASE_NAME=${USERNAME}"-${CLIENT_NAME}-lease"
echo LEASE_NAME $LEASE_NAME >> $CLIENT_INFO_FILE

NODE_MIN=1
NODE_MAX=1

NODE_TYPE='compute_haswell'
#NODE_TYPE='compute_skylake'

echo Creating Lease ${LEASE_NAME}
blazar lease-create \
   --physical-reservation min=${NODE_MIN},max=${NODE_MAX},resource_properties='["=", "$node_type", "'${NODE_TYPE}'"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${LEASE_NAME}

fi

echo LEASE_NAME: $LEASE_NAME
cat  $CLIENT_INFO_FILE

pruth-cici-universal-lease2
LEASE_NAME: pruth-cici-universal-lease2

CLIENT_NODE_NAME pruth-client1
CLIENT_STACK_NAME pruth-client1


In [4]:
RESERVATION_ID=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION_ID $RESERVATION_ID

RESERVATION_RESOURCE_ID=`blazar lease-show --format value -c reservations ${RESERVATION_ID} | grep \"id\" | cut -d "\"" -f 4`
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

#blazar lease-show ${RESERVATION_ID}

#Poll for node to be ready for ssh
while [[ true ]]
do 
	RESERVATION_STATUS=`blazar lease-show --format value -c status ${RESERVATION_ID}`
    
    if [[ "${RESERVATION_STATUS}" == "ACTIVE" ]]; then break; fi
    
    echo Reservation is ${RESERVATION_STATUS}. retrying in 5 seconds.
    sleep 5
done

echo RESERVATION_STATUS ${RESERVATION_STATUS}

echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID >> $CLIENT_INFO_FILE

cat $CLIENT_INFO_FILE


RESERVATION_ID 480a3fde-d50b-40d8-b48b-781729342b20
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc
RESERVATION_STATUS ACTIVE

CLIENT_NODE_NAME pruth-client1
CLIENT_STACK_NAME pruth-client1
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc


#### Start ExoGENI Client

In [ ]:
RESERVATION_RESOURCE_ID=`cat $CLIENT_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`

CLIENT_NETWORK='sharednet1'
CLIENT_NODE_NAME=`cat $CLIENT_INFO_FILE | grep CLIENT_NODE_NAME | cut -d " " -f 2`
CLIENT_STACK_NAME=`cat $CLIENT_INFO_FILE | grep CLIENT_STACK_NAME | cut -d " " -f 2`
GIT_BRANCH='tridentcom'
SLICE_NAME=${USERNAME}-${CLIENT_NAME}
IP_PREFIX='192.168.42.1/24'
CLIENT_SITE='UFL'
SAFE_KEYPAIR='${USERNAME}'

#IMAGE='CC-CentOS7'
IMAGE='CICI-SAFE.v1'

echo Creating SDX server. This will take several minutes!
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image $IMAGE \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${RESERVATION_RESOURCE_ID} \
                        --security-group default  \
                        --nic net-id=${CLIENT_NETWORK} \
                        --wait \
                        ${CLIENT_NODE_NAME}



#echo Creating SDX client. This will take several minutes!
#openstack stack create  --max-width 80 \
#        --template ${WORKING_DIR}/ExoGENI-Client/client.yaml \
#        --parameter "key_name=${SSH_KEY_NAME}" \
#        --parameter "reservation_id=${RESERVATION_RESOURCE_ID}" \
#--parameter "network_name=${CLIENT_NETWORK}" \
#        --parameter "client_name=${CLIENT_NODE_NAME}" \
#        --parameter "github_branch=${GIT_BRANCH}" \
#        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
#        --parameter "slice_name=${SLICE_NAME}"\
#        --parameter "ip_prefix=${IP_PREFIX}"\
#        --parameter "client_site=${CLIENT_SITE}"\
#        --parameter "safe_keypair=${SAFE_KEYPAIR}"\
#        --parameter "sdx_server=${SDX_PUBLIC_IP}:8888"\
#        --wait \
#        ${CLIENT_STACK_NAME}

Creating SDX server. This will take several minutes!


In [ ]:
# Wait until server is ready before continuing
server_status=""
while [[ $server_status != "ACTIVE" ]]
do
   sleep 5
   echo Getting status for $CLIENT_NODE_NAME
   server_status=$(openstack server list --format value -c Status --name "$CLIENT_NODE_NAME")
   echo status $server_status
done

In [ ]:
# Request a public floating IP (in the 'public' network)
CLIENT_PUBLIC_IP=$(openstack floating ip create public --format value -c floating_ip_address)

echo CLIENT_PUBLIC_IP $CLIENT_PUBLIC_IP

# Assign a public floating IP to $USER-first-server
openstack server add floating ip "$CLIENT_NODE_NAME" "$CLIENT_PUBLIC_IP"
echo CLIENT_PUBLIC_IP $CLIENT_PUBLIC_IP >> $CLIENT_INFO_FILE

In [ ]:
echo  Client creation complete! 
#echo ${CLIENT_NODE_NAME} status `openstack server show  --format value -c status ${CLIENT_NODE_NAME}`
CLIENT_PUBLIC_IP=`cat $CLIENT_INFO_FILE | grep CLIENT_PUBLIC_IP | cut -d " " -f 2`

echo The client public IP is $CLIENT_PUBLIC_IP

#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo SDX node ready for ssh

In [ ]:
CLIENT_PUBLIC_IP=`cat $CLIENT_INFO_FILE | grep CLIENT_PUBLIC_IP | cut -d " " -f 2`

#Copy GENI pem file to node
REMOTE_GENI_CERT=/home/cc/geni.pem
GENI_CERT=${HOME}/work/geni-pruth.pem

echo Copying GENI cert to cc@$CLIENT_PUBLIC_IP:~/geni.pem
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    ${GENI_CERT} \
    cc@${CLIENT_PUBLIC_IP}:${REMOTE_GENI_CERT}
echo Done

In [ ]:
SDX_PUBLIC_IP=`cat $SDX_INFO_FILE | grep SDX_PUBLIC_IP | cut -d " " -f 2`
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

REMOTE_SCRIPT=/home/cc/script.sh
LOCAL_SCRIPT=${WORKING_DIR}/riak_script.sh

cat << EOF > $LOCAL_SCRIPT
#!/bin/bash
sudo sh -c 'echo "PermitRootLogin yes" >>/etc/ssh/sshd_config'
#sudo yum install -y docker vim mlocate git maven
sudo systemctl start docker

echo Start SAFE Server in Container
#sudo docker pull yaoyj11/safeserver-v7
sudo docker run -i -t -d -p 7777:7777 -h safe --name safe yaoyj11/safeserver-v7
sudo docker exec -d safe /bin/bash -c  "cd /root/safe;export SBT_HOME=/opt/sbt-0.13.12;export SCALA_HOME=/opt/scala-2.11.8;sed -i 's/RIAKSERVER/$RIAK_PUBLIC_IP/g' safe-server/src/main/resources/application.conf;./prdn.sh"

echo Create ssh key
sudo sh -c 'ssh-keygen -t rsa -b 4096  -P "" -f "/root/.ssh/id_rsa"  -q'
sudo sh -c 'cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys2'
sudo sh -c 'chmod 600 ~/.ssh/authorized_keys2'

echo Boot Script Done!
EOF

echo Copying script to cc@$SDX_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_SCRIPT \
    cc@${SDX_PUBLIC_IP}:${REMOTE_SCRIPT}
echo Done

echo Running script
ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} "chmod +x "$REMOTE_SCRIPT" && "$REMOTE_SCRIPT

echo Done

#### Create SAFE keypair (STEP 1)

In [ ]:
SAFE_KEYPAIR=$CLIENT_NAME
CLIENT_PUBLIC_IP=`cat $CLIENT_INFO_FILE | grep CLIENT_PUBLIC_IP | cut -d " " -f 2`

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    'sudo docker exec -itd safe /bin/bash /root/safe_keygen.sh '${SAFE_KEYPAIR}' /root/safe/safe-server/src/main/resources/prdnsmall'


#get principalId (i.e. hash of public key)
res=$(curl http://${CLIENT_PUBLIC_IP}:7777/whoami \
     -H "Content-Type:application/json" \
     -d '{"principal":"'${SAFE_KEYPAIR}'","methodParams":[]}'
     )
principalId=$(echo $res | cut -d"'" -f 2)

#post 
res=$(curl http://${CLIENT_PUBLIC_IP}:7777/postIdSet \
    -H "Cntent-Type:application/json" \
    -d '{"principal":"'${SAFE_KEYPAIR}'","methodParams":["'${SAFE_KEYPAIR}'"]}'
    }
echo $res


echo principalId for ${SAFE_KEYPAIR} is $principalId


#### (STEP 2) -- Then goto "authority.ipynb"

In [ ]:
REMOTE_WORKING_DIR=/home/cc
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin
# init safe sets for the new keypair, post policies
echo ${principalId}
echo ${CLINET_PUBLIC_IP}
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/AuthorityMock init ${principalId} tag0 ${CLIENT_PUBLIC_IP}"

#### Use AuorityMock to make Delegations
##### Copy and paste those delegation tokens and add those tokens to safesets of the new keypair

#### (STEP 4)

In [ ]:
# Copy and paste each line from the output of authority to P
PARAMS='passDelegation X5NOWz1Ul_ZnisOhkRoyNWH0syvNVGVU9i-NOKky5Y4= User'
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/AuthorityMock update ${principalId} ${PARAMS} ${CLIENT_PUBLIC_IP}"

#### (STEP 5)

In [ ]:
LOCAL_CLIENT_CONFIG=${WORKING_DIR}/client1.conf

REMOTE_CLIENT_CONFIG=/home/cc/client1.conf
REMOTE_WORKING_DIR=/home/cc
REMOTE_BIN_DIR=${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/target/appassembler/bin

cat << EOF > $LOCAL_CLIENT_CONFIG
config {
slicename="${SLICE_NAME}"
type="client"
sshkey="/root/.ssh/id_rsa"
safekey="${SAFE_KEYPAIR}"
safeserver="127.0.0.1"
safe=true
serverinslice=false
exogenipem="${REMOTE_GENI_CERT}"
exogenism="https://geni.renci.org:11443/orca/xmlrpc"
scriptsdir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/scripts/"
resourcedir="${REMOTE_WORKING_DIR}/CICI-SAFE/SDX-Simple/SAFE_SDX/src/main/resources/"
ipprefix="${IP_PREFIX}"
serverurl="http://${SDX_PUBLIC_IP}:8888/"
routersite="${CLIENT_SITE}"
}
EOF

echo Copying SDX config to cc@$CLIENT_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_CLIENT_CONFIG \
    cc@${CLIENT_PUBLIC_IP}:${REMOTE_CLIENT_CONFIG}
echo Done

In [ ]:
echo Start Client Slice

echo $REMOTE_CLIENT_CONFIG

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    sudo ${REMOTE_BIN_DIR}/SafeSdxClientSliceServer -c $REMOTE_CLIENT_CONFIG

echo "Slice creation done"
#cd ${HOME}/work/CICI-SAFE/SDX-Simple
#./scripts/build.sh
#./scripts/createclientslice.sh -c $LOCAL_CLIENT_CONFIG

#### Client 1 stitch to SDX

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e 'stitch CNode1 192.168.10.2 192.168.10.1/24'"

#### Client 1 advertise its IP prefix

In [ ]:
ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e 'route 192.168.10.1/24 192.168.10.2'"


#### Connect client 1 and client 2

In [ ]:
CLIENT_CMD="\'link 192.168.10.1/24 192.168.20.1/24\'"

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e ${CLIENT_CMD}"

#### Undo stitching

In [ ]:
CLIENT_CMD="\'unstitch CNode1\'"

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CLIENT_PUBLIC_IP} \
    "sudo ${REMOTE_BIN_DIR}/SafeSdxExogeniClient -c ${REMOTE_CLIENT_CONFIG} -e ${CLIENT_CMD}"

## Clean up your resources

In [ ]:
CLIENT_STACK_NAME=`cat $CLIENT_INFO_FILE | grep CLIENT_STACK_NAME | cut -d " " -f 2`
openstack stack delete --wait -y $CLIENT_STACK_NAME

LEASE_NAME=`cat $CLIENT_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
blazar lease-delete $LEASE_NAME

cat $CLIENT_INFO_FILE
mv $CLIENT_INFO_FILE ${CLIENT_INFO_FILE}.old